<h4 align ='center'>Kushagra Singh | Infosys Project</h4>
<h4 align ='center'>Using MNIST Dataset and MLP & CNN models like LeNet5</h4>
<br>

In [1]:
pip install torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torchvision
from torchvision import datasets

torchvision.datasets.MNIST
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:10<00:00, 922131.66it/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 59678.93it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:42<00:00, 39013.73it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4597135.32it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

